#Environment Setup


In [ ]:
!pip install gymnasium
!pip install gymnasium[atari]

In [ ]:
!pip install gymnasium[accept-rom-license]

In [ ]:
!pip install "stable-baselines3[extra]>=2.0.0a4"

In [ ]:
%load_ext tensorboard

In [ ]:
import gymnasium as gym
import torch
# from gym.wrappers import RecordVideo

In [ ]:
env = gym.make('ALE/MsPacman-v5', render_mode=None)
# env = RecordVideo(env, video_folder='./video', episode_trigger=lambda x: True)

observation = env.reset()

In [ ]:
def reward_fn(current_reward, life_current, life_next, obs_current, obs_next):
    extra_reward = 0
    if life_current < life_next:
      extra_reward -= 10
    if (obs_current[0] == obs_next[0]).all():
      extra_reward -= 1
    if current_reward == 10:
      extra_reward += 10
    return extra_reward

In [ ]:
def img_preprocess(frame):
  grayscale_img = np.mean(frame, axis=2)
  resize_img = cv2.resize(grayscale_img, (144, 144), interpolation=cv2.INTER_AREA)
  return resize_img.astype(np.float32)

In [ ]:
actions = set()
steps = 0
life_current = 3
reward = 0
rewards = []
done = False
observation_current = img_preprocess(env.reset()[0])
for _ in range(1000):
    action = env.action_space.sample()
    actions.add(action)
    print(observation_current.shape)
    print(type(observation_current))
    observation_next, current_reward, done, info, check = env.step(action)
    life_next = check["lives"]
    reward += (current_reward + reward_fn(current_reward, life_current, life_next, observation_current, img_preprocess(observation_next)))
    # print(info)
    if(current_reward==10):
      print("10 reward!!!")
    print("cummulative reward", reward)
    observation_current = img_preprocess(observation_next)
    steps+=1
    life_current = life_next
    if done and check["lives"]==0:
        observation = env.reset()
        break
env.close()
print(steps)

In [ ]:
actions

In [ ]:
observation[0].shape

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

#Replay Buffer


In [ ]:
from collections import namedtuple
from collections import deque
import torch.nn as nn
import numpy as np
from torch.utils.tensorboard import SummaryWriter
import datetime
from typing import Tuple
from numpy.random import binomial
from numpy.random import choice
import numpy.random as nr
import torch.nn.functional as F
import cv2

Tensor = torch.DoubleTensor
torch.set_default_tensor_type(Tensor)
Transitions = namedtuple('Transitions', ['obs', 'action', 'reward', 'next_obs', 'done'])

In [ ]:
import random
from collections import deque

class ReplayBuffer:
    def __init__(self, capacity, state_shape, action_shape, device='cpu'):
        self.capacity = capacity
        self.device = device
        self.position = 0
        self.buffer = {
            'states': torch.empty((capacity, *state_shape), dtype=torch.float32, device=device),
            'actions': torch.empty((capacity, action_shape), dtype=torch.long, device=device),
            'rewards': torch.empty((capacity, 1), dtype=torch.float32, device=device),
            'next_states': torch.empty((capacity, *state_shape), dtype=torch.float32, device=device),
            'dones': torch.empty((capacity, 1), dtype=torch.float32, device=device)
        }
        self.size = 0

    def add(self, state, action, reward, next_state, done):
        self.buffer['states'][self.position] = torch.tensor(state, device=self.device)
        self.buffer['actions'][self.position] = torch.tensor(action, device=self.device)
        self.buffer['rewards'][self.position] = torch.tensor([reward], device=self.device)
        self.buffer['next_states'][self.position] = torch.tensor(next_state, device=self.device)
        self.buffer['dones'][self.position] = torch.tensor([done], device=self.device)

        self.position = (self.position + 1) % self.capacity
        if self.size < self.capacity:
            self.size += 1

    def sample(self, batch_size):
        indices = torch.randint(0, self.size, (batch_size,), device=self.device)
        batch = {k: v[indices] for k, v in self.buffer.items()}
        return batch

    def __len__(self):
        return self.size

In [ ]:
buffer_capacity = 1000
batch_size = 32
state_shape = (144, 144)
action_shape = 9

replay_buffer = ReplayBuffer(capacity=buffer_capacity, state_shape=state_shape, action_shape=action_shape, device=device)

#DQN

In [ ]:
class QNetwork(nn.Module):
    def __init__(self, dim_action):
        super(QNetwork, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.25)
        self.fc1 = nn.Linear(128 * 18 * 18, 512)
        self.fc2 = nn.Linear(512, dim_action)

        # self.apply(self._initialize_weights)

    def forward(self, x):
        # print(x.shape)
        x = x.unsqueeze(0)
        x = x.view(x.shape[1], x.shape[0], x.shape[2], x.shape[3])
        # print(x.shape)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 18 * 18)  # Flatten layer
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

    def _initialize_weights(self, m):
      if isinstance(m, nn.Conv2d):
          nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
          if m.bias is not None:
              nn.init.constant_(m.bias, 0)
      elif isinstance(m, nn.BatchNorm2d):
          nn.init.constant_(m.weight, 1)
          nn.init.constant_(m.bias, 0)
      elif isinstance(m, nn.Linear):
          nn.init.xavier_normal_(m.weight)
          nn.init.constant_(m.bias, 0)

In [ ]:
Q = QNetwork(dim_action=9)

In [ ]:
x = torch.rand((144, 144))
print(x.shape)

In [ ]:
with torch.no_grad():
  val = Q.forward(x.unsqueeze(0))
print(val)

In [ ]:
class DQN:
    def __init__(self, config):

        torch.manual_seed(config['seed'])

        self.lr = config['lr']  # learning rate
        self.C = config['C']  # copy steps
        self.eps_len = config['eps_len']  # length of epsilon greedy exploration
        self.eps_max = config['eps_max']
        self.eps_min = config['eps_min']
        self.discount = config['discount']  # discount factor
        self.batch_size = config['batch_size']  # mini batch size

        self.dim_action = config['dim_action']

        self.device = config['device']

        self.Q = QNetwork(dim_action=self.dim_action)
        self.Q_tar = QNetwork(dim_action=self.dim_action)

        self.Q = self.Q.to(self.device)
        self.Q_tar = self.Q_tar.to(self.device)

        self.optimizer_Q = torch.optim.Adam(self.Q.parameters(), lr=self.lr)
        self.training_step = 0

    def update(self, buffer):
        t = buffer.sample(self.batch_size)

        s = t["states"].to(self.device)
        a = t["actions"].to(self.device)
        r = t["rewards"].to(self.device)
        sp = t["next_states"].to(self.device)
        done = t["dones"].to(self.device)

        self.training_step += 1
        current_q_values = self.Q.forward(s.double()).gather(1, a)

        with torch.no_grad():
            expected_q_values = r + self.discount * torch.max(self.Q_tar.forward(sp.double()), axis=1, keepdim=True)[0] * (1-done)

        # print("current:", current_q_values.shape)
        # print(current_q_values)
        # print("expected:", expected_q_values.shape)
        # print(expected_q_values)
        loss = nn.functional.mse_loss(current_q_values.mean(dim=1, keepdim=True),expected_q_values)

        self.optimizer_Q.zero_grad()
        loss.backward()
        self.optimizer_Q.step()

        if self.training_step % self.C == 0:
            self.Q_tar.load_state_dict(self.Q.state_dict())

    def act_probabilistic(self, observation: torch.Tensor):
        # epsilon greedy:
        # print("Obs shape: ", observation.shape)
        first_term = self.eps_max * (self.eps_len - self.training_step) / self.eps_len
        eps = max(first_term, self.eps_min)

        explore = binomial(1, eps)

        if explore == 1:
            a = choice(self.dim_action)
        else:
            self.Q.eval()
            with torch.no_grad():
              Q = self.Q(observation.unsqueeze(0).double())
            val, a = torch.max(Q, axis=1)
            a = a.item()
        return a


In [ ]:
config = {
    'dim_action': 9,  # Q network output
    'lr': 0.0005,  # learning rate
    'C': 60,  # copy steps
    'discount': 0.99,  # discount factor
    'batch_size': 32,
    'replay_buffer_size': 100000,
    'eps_min': 0.01,
    'eps_max': 1.0,
    'eps_len': 4000,
    'seed': 1,
    'device':device
}

In [ ]:
print(device)

In [ ]:
dqn = DQN(config)
train_writer = SummaryWriter(log_dir='tensorboard/dqn')

In [ ]:
steps = 0  # total number of steps
rewards = []
for i_episode in range(500):
    observation = env.reset()[0]
    done = 0
    t = 0  # time steps within each episode
    ret = 0.  # episodic return
    life_current = 3
    for _ in range(1000):
        observation_tensor = torch.tensor(img_preprocess(observation)).to(device)
        action = dqn.act_probabilistic(observation_tensor)  # take action

        observation_next, current_reward, done, info, check = env.step(action)
        life_next = check["lives"]

        reward = current_reward + reward_fn(current_reward, life_current, life_next, img_preprocess(observation), img_preprocess(observation_next))
        # print(reward)
        replay_buffer.add(state=observation_tensor,  # put the transition to memory
                             action=torch.from_numpy(np.array([action])),
                             reward=torch.from_numpy(np.array([reward])),
                             next_state=torch.from_numpy(img_preprocess(observation_next)),
                             done=done)

        dqn.update(replay_buffer)  # agent learn

        t += 1
        steps += 1
        observation = observation_next
        ret += reward
        life_current = life_next
        if done and life_current==0:
            print("Episode {} finished after {} timesteps".format(i_episode, t+1))
            rewards.append(ret)
            break

        train_writer.add_scalar('Performance/episodic_return', ret, i_episode)  # plot

    print(f"Return in episode {i_episode} : {ret}")

env.close()
train_writer.close()

In [ ]:
len(rewards)

In [ ]:
%tensorboard --logdir='tensorboard/dqn'

#DDPG

In [ ]:
class ActorNet(nn.Module):
    def __init__(self, dim_action):
        super(ActorNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.25)
        self.fc1 = nn.Linear(128 * 18 * 18, 512)
        self.fc2 = nn.Linear(512, dim_action)

        self.apply(self._initialize_weights)

    def forward(self, x):
        # print(x.shape)
        x = x.unsqueeze(0).double()
        # print(x.shape)
        x = x.view(x.shape[1], x.shape[0], x.shape[2], x.shape[3])
        # print(x.shape)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 18 * 18)  # Flatten layer
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return torch.argmax(x, dim=-1)

    def _initialize_weights(self, m):
      if isinstance(m, nn.Conv2d):
          nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
          if m.bias is not None:
              nn.init.constant_(m.bias, 0)
      elif isinstance(m, nn.BatchNorm2d):
          nn.init.constant_(m.weight, 1)
          nn.init.constant_(m.bias, 0)
      elif isinstance(m, nn.Linear):
          nn.init.xavier_normal_(m.weight)
          nn.init.constant_(m.bias, 0)

In [ ]:
class QCriticNet(nn.Module):
    def __init__(self, dim_action):
        super(QCriticNet, self).__init__()
        self.dim_action = dim_action
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.25)
        self.fc1 = nn.Linear(128 * 18 * 18+dim_action, 512)
        self.fc2 = nn.Linear(512, dim_action)

        self.apply(self._initialize_weights)

    def forward(self, x, action):
        # print(x.shape)
        x = x.unsqueeze(0).double()
        x = x.view(x.shape[1], x.shape[0], x.shape[2], x.shape[3])
        # print(x.shape)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 18 * 18)  # Flatten layer
        # print("action shape in QNet(before mean conversion): ",action.shape)
        # print(action)
        if(len(action.shape)==2):
          # print("2d action flaged!!!")
          action = action[:,0]
        # print("action shape in QNet(after mean conversion): ",action.shape)
        ohe_action_space = F.one_hot(action.to(int), num_classes=9).squeeze(1)
        # print("ohe shape in QNet(after ohe conversion): ", ohe_action_space.shape)
        # print("x shape in QNet: ", x.shape)
        #   print("ohe shape in QNet(after ohe conversion then this internal transformation): ", ohe_action_space.shape)
        # print(ohe_action_space)
        x = torch.cat((x, ohe_action_space), dim=-1)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return torch.argmax(x, dim=-1)

    def _initialize_weights(self, m):
      if isinstance(m, nn.Conv2d):
          nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
          if m.bias is not None:
              nn.init.constant_(m.bias, 0)
      elif isinstance(m, nn.BatchNorm2d):
          nn.init.constant_(m.weight, 1)
          nn.init.constant_(m.bias, 0)
      elif isinstance(m, nn.Linear):
          nn.init.xavier_normal_(m.weight)
          nn.init.constant_(m.bias, 0)

In [ ]:
class DDPG(nn.Module):
    def __init__(self, config):
        super(DDPG,self).__init__()
        torch.manual_seed(config['seed'])

        self.lr_actor = config['lr_actor']  # learning rate
        self.lr_critic = config['lr_critic']
        self.smooth = config['smooth']  # smoothing coefficient for target net
        self.discount = config['discount']  # discount factor
        self.batch_size = config['batch_size']  # mini batch size
        self.sig = config['sig']  # exploration noise

        self.dim_action = config['dim_action']

        self.device = config['device']

        self.actor = ActorNet(dim_action=self.dim_action).to(self.device)
        self.Q = QCriticNet(dim_action=self.dim_action).to(self.device)
        self.actor_tar = ActorNet(dim_action=self.dim_action).to(self.device)
        self.Q_tar = QCriticNet(dim_action=self.dim_action).to(self.device)

        self.optimizer_actor = torch.optim.Adam(self.actor.parameters(), lr=self.lr_actor)
        self.optimizer_Q = torch.optim.Adam(self.Q.parameters(), lr=self.lr_critic)

    def update(self, buffer):
        t = buffer.sample(self.batch_size)

        obs = t["states"].to(self.device)
        action = t["actions"].to(self.device)
        reward = t["rewards"].to(self.device)
        next_obs = t["next_states"].to(self.device)
        done = t["dones"].to(self.device)

        # print("area 1 start")
        with torch.no_grad():
          y = reward + self.discount*(1-done)*self.Q_tar(next_obs, self.actor_tar(next_obs))


        # print("area 2 start")
        loss_Q = torch.mean((self.Q(obs,action)-y)**2)
        loss_Q.requires_grad = True
        # print("area 2 end")
        self.optimizer_Q.zero_grad()
        loss_Q.backward()
        self.optimizer_Q.step()
        # print("area 3 start")
        objective_actor = torch.mean(self.Q(obs,self.actor(obs).double()))
        # print("area 3 end")
        loss_actor = -objective_actor

        self.optimizer_actor.zero_grad()
        loss_actor.backward()
        self.optimizer_actor.step()

        for target_param, param in zip(self.Q_tar.parameters(), self.Q.parameters()):
          target_param.data.copy_(target_param.data*(self.smooth)+param.data*(1-self.smooth))

        for target_param, param in zip(self.actor_tar.parameters(), self.actor.parameters()):
          target_param.data.copy_(target_param.data*(self.smooth)+param.data*(1-self.smooth))

    def act_probabilistic(self, obs: torch.Tensor):
        self.actor.eval()
        exploration_noise = torch.normal(torch.zeros(size=(self.dim_action,)), self.sig).to(self.device)
        a = self.actor(obs.unsqueeze(0)) + exploration_noise
        self.actor.train()
        return torch.argmax(a, dim=-1)

    def act_deterministic(self, obs: torch.Tensor):
        self.actor.eval()
        a = self.actor(obs)
        self.actor.train()
        return a

In [ ]:
config = {
    'dim_action': 9,
    'lr_actor': 0.001,
    'lr_critic': 0.005,
    'smooth': 0.99,
    'discount': 0.99,
    'sig': 0.01,
    'batch_size': 32,
    'replay_buffer_size': 20000,
    'seed': 1,
    'max_episode': 500,
    'device':device
}

In [ ]:
ddpg = DDPG(config)
train_writer = SummaryWriter(log_dir='tensorboard/ddpg')

In [ ]:
steps = 0  # total number of steps
rewards = []
for i_episode in range(300):
    observation = env.reset()[0]
    done = 0
    t = 0  # time steps within each episode
    ret = 0.  # episodic return
    life_current = 3
    for _ in range(1000):
        observation_tensor = torch.tensor(img_preprocess(observation)).to(device)
        action = ddpg.act_probabilistic(observation_tensor).cpu().item()  # take action

        observation_next, current_reward, done, info, check = env.step(action)
        life_next = check["lives"]

        reward = current_reward + reward_fn(current_reward, life_current, life_next, img_preprocess(observation), img_preprocess(observation_next))
        # print(reward)
        replay_buffer.add(state=observation_tensor,  # put the transition to memory
                             action=torch.from_numpy(np.array([action])),
                             reward=torch.from_numpy(np.array([reward])),
                             next_state=torch.from_numpy(img_preprocess(observation_next)),
                             done=done)

        ddpg.update(replay_buffer)  # agent learn

        t += 1
        steps += 1
        observation = observation_next
        ret += reward
        life_current = life_next
        if done and life_current==0:
            print("Episode {} finished after {} timesteps".format(i_episode, t+1))
            rewards.append(ret)
            break

        train_writer.add_scalar('Performance/episodic_return', ret, i_episode)  # plot

    print(f"Return in episode {i_episode} : {ret}")

env.close()
train_writer.close()

In [ ]:
%tensorboard --logdir='tensorboard/ddpg'

#SAC

In [ ]:
class ActorNet(nn.Module):
    def __init__(self, dim_action: int, device):

        super(ActorNet, self).__init__()

        self.dim_action = dim_action
        self.device = device

        self.ln2pi = torch.tensor(np.log(2*np.pi)).to(self.device)

        self.convLayer1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=6, stride=4, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=3)
        )
        self.convLayer2 = nn.Sequential(
            nn.Conv2d(32, 28, kernel_size=5, padding=1),
            nn.ReLU()
        )

        self.fc = nn.Linear(28*15*11, 512)

        self.mu = nn.Linear(512, self.dim_action)
        self.log_sig = nn.Linear(512, self.dim_action)

        self.apply(self._initialize_weights)

    def forward(self, observation: torch.Tensor):
        x = observation.double()
        # print("input ready")
        # print(x.shape)
        x = x.view(x.shape[0],x.shape[3],x.shape[1],x.shape[2])
        # print(x.shape)
        x = self.convLayer1(x)
        # print("conv layer 1 op ready")
        x = self.convLayer2(x)
        # print("conv op:", x.shape)
        x = x.reshape(-1, 28*15*11)
        # print("conv op:", x.shape)
        x = F.relu(self.fc(x))
        mu_dist = self.mu(x)
        sig_dist = torch.exp(self.log_sig(x))
        # print(x.shape)
        u = mu_dist + sig_dist * torch.normal(torch.zeros(size=mu_dist.shape), 1).to(self.device)
        a = torch.tanh(u)
        logProbu = -1/2 * (torch.sum(torch.log(sig_dist**2), dim=1, keepdims=True).to(self.device) +
                           torch.sum((u-mu_dist)**2/sig_dist**2, dim=1, keepdims=True) +
                           a.shape[1]*self.ln2pi.to(self.device))
        logProba = logProbu - torch.sum(torch.log(1 - a ** 2 + 0.000001), dim=1, keepdims=True)

        return a, logProba, torch.tanh(mu_dist)

    def _initialize_weights(self, m):
      if isinstance(m, nn.Conv2d):
          nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
          if m.bias is not None:
              nn.init.constant_(m.bias, 0)
      elif isinstance(m, nn.BatchNorm2d):
          nn.init.constant_(m.weight, 1)
          nn.init.constant_(m.bias, 0)
      elif isinstance(m, nn.Linear):
          nn.init.xavier_normal_(m.weight)
          nn.init.constant_(m.bias, 0)

In [ ]:
class SAC(nn.Module):
    def __init__(self, config):
        super(SAC,self).__init__()
        torch.manual_seed(config['seed'])

        self.lr = config['lr']  # learning rate
        self.smooth = config['smooth']  # smoothing coefficient for target net
        self.discount = config['discount']  # discount factor
        self.alpha = config['alpha']  # temperature parameter in SAC
        self.batch_size = config['batch_size']  # mini batch size

        self.dim_action = config['dim_action']

        self.device = config['device']

        self.actor = ActorNet(dim_action=self.dim_action, device = self.device).to(self.device)
        self.Q1 = QCriticNet(dim_action=self.dim_action).to(self.device)
        self.Q2 = QCriticNet(dim_action=self.dim_action).to(self.device)
        self.Q1_tar = QCriticNet(dim_action=self.dim_action).to(self.device)
        self.Q2_tar = QCriticNet(dim_action=self.dim_action).to(self.device)

        self.optimizer_actor = torch.optim.Adam(self.actor.parameters(), lr=self.lr)
        self.optimizer_Q1 = torch.optim.Adam(self.Q1.parameters(), lr=self.lr)
        self.optimizer_Q2 = torch.optim.Adam(self.Q2.parameters(), lr=self.lr)

    def update(self, buffer):
        # sample from replay memory
        t = buffer.sample(self.batch_size)

        obs = t["states"].to(self.device)
        action = t["actions"].to(self.device)
        reward = t["rewards"].to(self.device)
        next_obs = t["next_states"].to(self.device)
        done = t["dones"].to(self.device)

        with torch.no_grad():
          next_action_sample, next_logProb_sample, next_mu_sample = self.actor(next_obs)
          Qp = torch.min(self.Q1_tar(next_obs,next_action_sample), self.Q2_tar(next_obs,next_action_sample))
          Q_target = reward + self.discount*(~done)*(Qp-self.alpha*next_logProb_sample)

        loss_Q1 = torch.mean((self.Q1(obs,action)-Q_target)**2)
        loss_Q2 = torch.mean((self.Q2(obs,action)-Q_target)**2)

        self.optimizer_Q1.zero_grad()
        loss_Q1.backward()
        self.optimizer_Q1.step()

        self.optimizer_Q2.zero_grad()
        loss_Q2.backward()
        self.optimizer_Q2.step()

        action_sample, logProb_sample, mu_sample = self.actor(obs)
        Q = torch.min(self.Q1(obs,action_sample), self.Q2(obs,action_sample))
        objective_actor = torch.mean(Q-self.alpha*logProb_sample)
        loss_actor = -objective_actor

        self.optimizer_actor.zero_grad()
        loss_actor.backward()
        self.optimizer_actor.step()

        for target_param, param in zip(self.Q1_tar.parameters(), self.Q1.parameters()):
          target_param.data.copy_(target_param.data*(1-self.smooth)+param.data*self.smooth)

        for target_param, param in zip(self.Q2_tar.parameters(), self.Q2.parameters()):
          target_param.data.copy_(target_param.data*(1-self.smooth)+param.data*self.smooth)

    def act_probabilistic(self, obs: torch.Tensor):
        self.actor.eval()
        a, logProb, mu = self.actor(obs)
        self.actor.train()
        return a

    def act_deterministic(self, obs: torch.Tensor):
        self.actor.eval()
        a, logProb, mu = self.actor(obs)
        self.actor.train()
        return mu

In [ ]:
config = {
    'dim_action': 2,
    'lr': 0.001,
    'smooth': 0.99,
    'discount': 0.99,
    'alpha': 0.2,
    'batch_size': 128,
    'replay_buffer_size': 20000,
    'seed': 1,
    'max_episode': 500,
    'device':device
}

In [ ]:
sac = SAC(config)
train_writer = SummaryWriter(log_dir='tensorboard/sac')

In [ ]:
class ReplayBuffer:
    def __init__(self, state_shape, action_dim, max_size=int(1e6)):
        self.max_size = max_size
        self.ptr = 0
        self.size = 0

        # Adjusted for 3D state tensors
        self.state = torch.zeros((max_size, *state_shape))
        self.action = torch.zeros(max_size, action_dim)  # Actions are probability distributions
        self.next_state = torch.zeros((max_size, *state_shape))
        self.reward = torch.zeros(max_size, 1)
        self.not_done = torch.zeros(max_size, 1)

    def add(self, state, action, next_state, reward, done):
        self.state[self.ptr] = torch.from_numpy(state)
        self.action[self.ptr] = torch.from_numpy(action)
        self.next_state[self.ptr] = torch.from_numpy(next_state)
        self.reward[self.ptr] = reward
        self.not_done[self.ptr] = 1. - done

        self.ptr = (self.ptr + 1) % self.max_size
        self.size = min(self.size + 1, self.max_size)

    def sample(self, batch_size):
        ind = torch.randint(0, self.size, (batch_size,), dtype=torch.long)

        return (
            self.state[ind],
            self.action[ind],
            self.next_state[ind],
            self.reward[ind],
            self.not_done[ind]
        )

In [ ]:
steps = 0
for i_episode in range(config['max_episode']):
    obs = env.reset()[0]
    done = False
    truncated = False
    t = 0
    ret = 0.
    while done is False and truncated is False:
        # env.render()

        obs_tensor = torch.tensor(obs).type(Tensor).to(device)

        action = sac.act_probabilistic(obs_tensor.unsqueeze(0)).detach().cpu().numpy()[0,:]
        # print(action[0])

        next_obs, reward, done, truncated,_ = env.step(int(action[0]))

        replay_buffer.add(state=obs_tensor,
                             action=torch.from_numpy(action),
                             reward=torch.from_numpy(np.array([reward/10.0])),
                             next_state=torch.from_numpy(next_obs).type(Tensor),
                             done=done)

        # sac.update(replay_buffer)

        t += 1
        steps += 1
        ret += reward

        obs = next_obs

        if done or truncated:
            print("Episode {} return {}".format(i_episode, ret))
    train_writer.add_scalar('Performance/episodic_return', ret, i_episode)

env.close()
train_writer.close()

In [ ]:
%tensorboard --logdir='tensorboard/sac'